## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSBBS'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
330
+-----------------+------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|          coeff_1|           coeff_2|           coeff_3|            coeff_4|elevation|   label|latitude|longitude|measurement|             res_1|              res_2|              res_3|              res_4|          res_mean|    station|   total_var|undefs|              vector|  year|
+-----------------+------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|8479.193692677072|2685.5045147058017| 1675.980173961427|-1122.00078

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [25]:
def getPDF(coeff_index):
    #extract longitude and latitude for each station
    sqlContext.registerDataFrameAsTable(df,'weather')
    #Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
    #          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
    Features='station, year, coeff_%d'%coeff_index
    Query="SELECT %s FROM weather"%Features
    print(Query)
    pdf = sqlContext.sql(Query).toPandas()
    return pdf
pdf = getPDF(1)

SELECT station, year, coeff_1 FROM weather


In [27]:
def getYearStationTable(coeff_index):
    pdf = getPDF(coeff_index)
    year_station_table=pdf.pivot(index='year', columns='station', values='coeff_%d'%coeff_index)
    return year_station_table
year_station_table = getYearStationTable(1)
year_station_table.head(10)

SELECT station, year, coeff_1 FROM weather


station,USC00040943,USC00041072,USC00041075,USC00041277,USC00041909,USC00041911,USC00042671,USC00043134,USC00043381,USC00043384,...,USC00263515,USC00265168,USC00265191,USC00266746,USC00267640,USC00267806,USC00268186,USC00268202,USC00268977,USC00269229
year,,,,,,,,,,,,,,,,,,,,,
1915.0,NaN,578.812443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1932.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-250.985145,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1933.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-25.387133,-73.003204,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1937.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-153.179937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1949.0,NaN,NaN,1287.347025,NaN,495.037312,NaN,NaN,505.860414,327.914840,1442.593435,...,-100.689112,NaN,274.460212,NaN,NaN,NaN,NaN,NaN,-244.434887,-218.014407
1950.0,NaN,NaN,-211.141635,NaN,NaN,NaN,NaN,-104.046408,-266.345816,64.856622,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1951.0,NaN,NaN,70.283381,NaN,NaN,NaN,NaN,-235.624175,NaN,-331.603288,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1952.0,NaN,NaN,NaN,NaN,NaN,472.033291,NaN,958.396164,-156.671405,911.505584,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [54]:
def estimateEffect(coeff_index):
    year_station_table = getYearStationTable(coeff_index)
    def RMS(Mat):
        return np.sqrt(np.nanmean(Mat**2))

    def MS(Mat):
        return np.nanmean(Mat**2)
    
    mean_by_year=np.nanmean(year_station_table,axis=1)
    mean_by_station=np.nanmean(year_station_table,axis=0)
    
    tbl_minus_year = year_station_table
    tbl_minus_station = year_station_table
    #for i in range(coeff_index):
    #    print "Minus %d"%i
    tbl_minus_year = (tbl_minus_year.transpose()-mean_by_year).transpose()
    tbl_minus_station = tbl_minus_station-mean_by_station

    print 'total RMS                   = %.2f'%RMS(year_station_table)
    print 'RMS removing mean-by-station= %.2f'%RMS(tbl_minus_station)
    print 'RMS removing mean-by-year   = %.2f'%RMS(tbl_minus_year)
    print '----------------------------------------------------'
    print 'total MS                   = %.2f'%MS(year_station_table)
    ms0 = MS(year_station_table)
    print 'MS removing mean-by-station= %.2f'%MS(tbl_minus_station)
    ms1 = MS(tbl_minus_station)
    print 'fraction explained = %.1f'%(100.-100.*ms1/ms0)
    print 'MS removing mean-by-year   = %.2f'%MS(tbl_minus_year)
    ms2 = MS(tbl_minus_year)
    print 'fraction explained = %.1f'%(100.-100.*ms2/ms0)
    print '----------------------------------------------------'


In [55]:
top_num_coeff = 4
for i in range(top_num_coeff):
    estimateEffect(i+1)

SELECT station, year, coeff_1 FROM weather
total RMS                   = 3524.55
RMS removing mean-by-station= 1746.91
RMS removing mean-by-year   = 2867.27
----------------------------------------------------
total MS                   = 12422473.71
MS removing mean-by-station= 3051699.85
fraction explained = 75.4
MS removing mean-by-year   = 8221246.26
fraction explained = 33.8
----------------------------------------------------
SELECT station, year, coeff_2 FROM weather
total RMS                   = 1174.86
RMS removing mean-by-station= 978.86
RMS removing mean-by-year   = 882.70
----------------------------------------------------
total MS                   = 1380301.03
MS removing mean-by-station= 958162.25
fraction explained = 30.6
MS removing mean-by-year   = 779160.94
fraction explained = 43.6
----------------------------------------------------
SELECT station, year, coeff_3 FROM weather
total RMS                   = 952.10
RMS removing mean-by-station= 910.75
RMS removing mea